In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5/
!pip install -r requirements.txt

In [6]:
!yolo checks



Ultralytics YOLOv8.2.42 рџљЂ Python-3.11.5 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4070, 12281MiB)
Setup complete вњ… (16 CPUs, 31.9 GB RAM, 791.9/926.1 GB disk)

OS                  Windows-10-10.0.19045-SP0
Environment         Windows
Python              3.11.5
Install             pip
RAM                 31.92 GB
CPU                 AMD Ryzen 7 5700X 8-Core Processor
CUDA                11.8

numpy               вњ… 1.24.3<2.0.0,>=1.23.5
matplotlib          вњ… 3.7.1>=3.3.0
opencv-python       вњ… 4.8.0.76>=4.6.0
pillow              вњ… 10.3.0>=7.1.2
pyyaml              вњ… 6.0>=5.3.1
requests            вњ… 2.32.3>=2.23.0
scipy               вњ… 1.10.1>=1.4.1
torch               вњ… 2.0.1>=1.8.0
torchvision         вњ… 0.15.2>=0.9.0
tqdm                вњ… 4.65.0>=4.64.0
psutil              вњ… 5.9.0
py-cpuinfo          вњ… 9.0.0
pandas              вњ… 1.5.3>=1.1.4
seaborn             вњ… 0.12.2>=0.11.0
ultralytics-thop    вњ… 2.0.0>=2.0.0


# Worklflow for Net_2

## Dataset convertionrom Label-Studio to YOLO

In [3]:
import json
import pandas as pd

In [4]:
path = 'C:/Users/User/Petr/Net_2/NEW_DATASET/annotation.json'

In [5]:
data = pd.read_json(path, orient='records')

In [6]:
import random
import shutil

max_points = 17

min_points = max_points

label_paths = {'train': 'C:/Users/User/Petr/Net_2/NEW_DATASET/Dataset_YOLO/labels/train',
               'val': 'C:/Users/User/Petr/Net_2/NEW_DATASET/Dataset_YOLO/labels/val'}

images_paths = {'train': 'C:/Users/User/Petr/Net_2/NEW_DATASET/Dataset_YOLO/images/train',
               'val': 'C:/Users/User/Petr/Net_2/NEW_DATASET/Dataset_YOLO/images/val'}

train_val = ['train', 'val']

for row in data.iterrows():
    name = row[1].data['img'].split('-')[1][:-4]

    kps = []
    for result in row[1].annotations[0]['result']:
        value = result['value']
        original_w = result['original_width']
        original_h = result['original_height']
        if result['from_name'] == 'keypoints':
            x, y = value['x'] / 100.0, value['y'] / 100.0
            kps.append((x, y, 2))
        else:
            x, y = value['x'] / 100.0, value['y'] / 100.0
            w, h = value['width'] / 100.0, value['height'] / 100.0
            rect = [x + w/2.0, y + h/2.0, w, h]
    while len(kps) < max_points:
        kps.append((0, 0, 0))

    folder = random.choices(train_val, weights=[0.75, 0.25])[0]
    shutil.copyfile(f'C:/Users/User/Petr/Net_2/NEW_DATASET/images/{name}.jpg', images_paths[folder] + f'/{name}.jpg')
    with open(label_paths[folder] + f'/{name}.txt', 'w') as file:
        file.write(f'0 {rect[0]} {rect[1]} {rect[2]} {rect[3]}')
        for point in kps:
            file.write(f' {point[0]} {point[1]} {point[2]}')


## Workflow

In [1]:
import clearml; clearml.browser_login()

In [2]:
%cd yolov5/

C:\Users\User\Petr\Net_4\Yolo\yolov5


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("C:/Users/User/Petr/Net_4/Yolo/yolov5/Corners/Corners_result45/weights/best.pt")  # load a pretrained model (recommended for training) ./Corners/Corners_result25/weights/best.pt

# Train the model
results = model.train(data="./datasets/Corners_RF/data.yaml", epochs=300, imgsz=640, batch=-1, patience=100, fraction=1.0, project='Corners', name='Corners_result', optimizer='SGD', lr0=0.05, lrf=0.01)

New https://pypi.org/project/ultralytics/8.2.71 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4070, 12281MiB)
engine\trainer: task=pose, mode=train, model=C:/Users/User/Petr/Net_4/Yolo/yolov5/Corners/Corners_result45/weights/best.pt, data=./datasets/Corners_RF/data.yaml, epochs=300, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Corners, name=Corners_result48, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fa

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/amodion/corners/5a476caefba34c779eb1cfd16950cde7



TensorBoard: Start with 'tensorboard --logdir Corners\Corners_result48', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
AMP: checks passed 
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070) 11.99G total, 0.58G reserved, 0.57G allocated, 10.85G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    69482247       263.9         2.202         148.4           nan        (1, 3, 640, 640)                    list
    69482247       527.8         3.683          43.7           nan        (2, 3, 640, 640)                    list
    69482247        1056         6.140         64.93           nan        (4, 3, 640, 640)                    list
    69482247        2111        11

train: Scanning C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\train\labels.cache... 1810 images, 5 backgroun

train: WARNING  C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\train\images\TEO_2_00538_Proto_crop_0_jpg.rf.2180fdb572ce6c73ad39ecffea9b3d0b.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.157]
train: WARNING  C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\train\images\TEO_2_00538_Proto_crop_0_jpg.rf.358a16248c0c13e7930ba1faa658c198.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.157]
train: WARNING  C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\train\images\TEO_2_00538_Proto_crop_0_jpg.rf.4661bd266efbe859b77f45a6f204190d.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.157]
train: WARNING  C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\train\images\TEO_2_00538_Proto_crop_0_jpg.rf.7575dc4ac4ebcd70d12b11e984cbd182.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.157]
train: WARNING  C:\Users

...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     43/300      4.44G     0.4929      1.881     0.4885     0.3349      1.156          4        640: 100%|██████████| 4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP

                   all         71         70      0.985          1      0.993      0.921      0.436      0.443      0.272      0.135



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     44/300      4.38G     0.4894      1.892     0.4825     0.3355      1.136         12        640:  94%|█████████▍| 4

In [6]:
from ultralytics import YOLO

model = YOLO("C:/Users/User/Petr/Net_4/Yolo/yolov5/Corners/Corners_result45/weights/best.pt")

pred = model.val(data='./datasets/Corners_RF/data.yaml', plots=True, split='test')

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4070, 12281MiB)
YOLOv8x-pose summary (fused): 287 layers, 69452727 parameters, 0 gradients, 263.2 GFLOPs


val: Scanning C:\Users\User\Petr\Net_4\Yolo\yolov5\datasets\Corners_RF\test\labels.cache... 16 images, 0 backgrounds, 0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP


                   all         16         16          1          1      0.995      0.978      0.438      0.438      0.465      0.439
Speed: 1.0ms preprocess, 31.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\pose\val8
2024-08-01 08:12:41,900 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###
